# Grid Point CMIP6 Retrieval and MET Conversion Notebook

## Overview

This notebook combines CMIP6 NetCDF data extraction with APSIM MET format conversion. It processes climate variables from NC files organized in `C:\Users\ibian\Desktop\ClimAdapt\CMIP6\{Model} {Scenario}` folders and converts them to MET format for APSIM simulations.

## File Structure

- **Input**: NC files in `C:\Users\ibian\Desktop\ClimAdapt\CMIP6\{Model} {Scenario}\` (e.g., `ACCESS CM2 SSP245`)
- **User Input**: Latitude and longitude coordinates (decimal degrees)
- **Output**: MET files and CSV files for the specified coordinate

## Variables Processed

The notebook processes 4 climate variables:

- **tasmax**: Daily maximum temperature (°C) → **maxt** in MET format
- **tasmin**: Daily minimum temperature (°C) → **mint** in MET format
- **pr**: Daily precipitation (mm) → **rain** in MET format
- **rsds**: Daily surface downwelling shortwave radiation (W/m²) → **radn** (MJ/m²) in MET format

**Note**: vp (vapor pressure) and code fields are left blank in the MET file

## MET Format Specifications

The MET format is used by APSIM for weather data input. It includes:

- **Required fields**: year, day, maxt (from tasmax), mint (from tasmin), rain (from pr), radn (from rsds)
- **radn field**: Required, converted from rsds (W/m² to MJ/m²)
- **Blank fields**: evap (evaporation), vp (vapor pressure), code (data quality code) - left blank
- **Metadata**: latitude, longitude, tav (annual average temperature), amp (annual amplitude)

## File Naming Conventions

- **Input NC files**: `{Model} {Scenario}\*{variable}*.nc` (e.g., `ACCESS CM2 SSP245\tasmax*.nc`)
- **Output MET files**: `{Model}_{Scenario}_{Lat}_{Lon}.met` (e.g., `ACCESS_CM2_SSP245_-31.75_117.60.met`)
- **Output CSV files**: `{Model}_{Scenario}_{Lat}_{Lon}_{variable}.csv` (e.g., `ACCESS_CM2_SSP245_-31.75_117.60_tasmax.csv`)

## Usage Instructions

1. Set the configuration parameters (Model, Scenario, output directory) in Section 1
2. Provide your target latitude and longitude coordinates in Section 2
3. Run all cells sequentially to extract data and create MET files
4. Output files will be saved with coordinate-based naming

## Coordinate Matching

The notebook finds the nearest grid point to your specified coordinates within a tolerance (default: 0.01 degrees ≈ 1.1 km). If the nearest point is outside tolerance, a warning will be displayed.

## Notes

- **CMIP6 data structure**: Data is organized by Model and Scenario in separate folders
- **Coordinate format**: Provide coordinates in decimal degrees (latitude: -90 to 90, longitude: -180 to 180)
- **Variable extraction**: 4 variables (tasmax, tasmin, pr, rsds) are extracted from NC files and saved as individual CSVs
- **MET conversion**: tasmax→maxt, tasmin→mint, pr→rain, rsds→radn
- **Unit conversions**: rsds (W/m²) is converted to radn (MJ/m²) by multiplying by 0.0864
- **Blank fields**: vp and code are left blank in MET format

## Section 1: Imports and Configuration

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import glob
import os
import time
import re
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Configuration
CMIP6_BASE_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6"
OUTPUT_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.45_117.55"  # Output directory for MET files
COORD_TOLERANCE = 0.01  # degrees (approximately 1.1 km)

# Model and Scenario - UPDATE THESE AS NEEDED
MODEL = "ACCESS CM2"  # e.g., "ACCESS CM2"
SCENARIO = "obs"   # e.g., "obs", "SSP245", or "SSP585"

# Date Range - UPDATE THESE AS NEEDED
START_YEAR = 1986   # Start year for MET file (e.g., 1985 for obs, 2035 for future scenarios, or None for all available data)
END_YEAR = 2014     # End year for MET file (e.g., 2014 for obs, 2064 for future, or None for all available data)

# Variables to process (5 variables)
# MET format mapping:
# - tasmax → maxt (maximum temperature)
# - tasmin → mint (minimum temperature)
# - pr → rain (precipitation)
# - rsds → radn (radiation, converted from W/m² to MJ/m²)
# - hurs → vp (vapor pressure, calculated using SILO method)
# Note: code is hardcoded to '222222'
VARIABLES = ['tasmax', 'tasmin', 'pr', 'rsds', 'hurs']

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Configuration loaded:")
print(f"  - CMIP6 Base Directory: {CMIP6_BASE_DIR}")
print(f"  - Output Directory: {OUTPUT_DIR}")
print(f"  - Model: {MODEL}")
print(f"  - Scenario: {SCENARIO}")
print(f"  - Date Range: {START_YEAR if START_YEAR is not None else 'start of data'} to {END_YEAR if END_YEAR is not None else 'end of data'}")
print(f"  - Coordinate Tolerance: {COORD_TOLERANCE} degrees")
print(f"  - Variables: {', '.join(VARIABLES)}")

Configuration loaded:
  - CMIP6 Base Directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6
  - Output Directory: C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.45_117.55
  - Model: ACCESS CM2
  - Scenario: obs
  - Date Range: 1986 to 2014
  - Coordinate Tolerance: 0.01 degrees
  - Variables: tasmax, tasmin, pr, rsds, hurs


## Section 2: User Coordinate Input

Provide the latitude and longitude coordinates for the grid point you want to extract.

In [3]:
# USER INPUT: Provide your target coordinates here
LATITUDE = -31.45   # Target latitude in decimal degrees (-90 to 90)
LONGITUDE = 117.55  # Target longitude in decimal degrees (-180 to 180)

# Validate coordinates
if not (-90 <= LATITUDE <= 90):
    raise ValueError(f"Latitude must be between -90 and 90. Provided: {LATITUDE}")
if not (-180 <= LONGITUDE <= 180):
    raise ValueError(f"Longitude must be between -180 and 180. Provided: {LONGITUDE}")

print(f"Target Coordinate:")
print(f"  Latitude: {LATITUDE:.6f}°")
print(f"  Longitude: {LONGITUDE:.6f}°")
print(f"  Model: {MODEL}")
print(f"  Scenario: {SCENARIO}")
print(f"  Tolerance: {COORD_TOLERANCE} degrees")

Target Coordinate:
  Latitude: -31.450000°
  Longitude: 117.550000°
  Model: ACCESS CM2
  Scenario: obs
  Tolerance: 0.01 degrees


## Section 3: NetCDF Extraction Function

In [4]:
def extract_daily_data_from_netcdf(netcdf_dir, variable, target_lat, target_lon, tolerance=0.01):
    """
    Extract daily time series data for a specific coordinate from NetCDF files.
    
    Parameters:
    -----------
    netcdf_dir : str
        Directory containing NetCDF files for the variable
    variable : str
        Variable name (tasmax, tasmin, pr, rsds)
    target_lat : float
        Target latitude
    target_lon : float
        Target longitude
    tolerance : float
        Coordinate matching tolerance in degrees
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: date, value
    """
    start_time = time.time()
    
    # Find all NetCDF files in the directory
    # Pattern 1: Files directly in the directory matching *{variable}*.nc
    nc_files = sorted(glob.glob(os.path.join(netcdf_dir, f"*{variable}*.nc")))
    
    # Pattern 2: Files in subdirectories named {variable}_* (e.g., pr_ACCESS CM2 SSP245)
    if len(nc_files) == 0:
        var_subdirs = glob.glob(os.path.join(netcdf_dir, f"{variable}_*"))
        for var_subdir in var_subdirs:
            if os.path.isdir(var_subdir):
                found_files = sorted(glob.glob(os.path.join(var_subdir, "*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(var_subdir)}/")
                    break
    
    # Pattern 2b: For rsds, also check for "rad_" folder (folder named "rad_" but files contain "rsds")
    # Example: rad_ACCESS CM2 SSP245/ contains files named *rsds*.nc
    if len(nc_files) == 0 and variable == 'rsds':
        rad_subdirs = glob.glob(os.path.join(netcdf_dir, "rad_*"))
        for rad_subdir in rad_subdirs:
            if os.path.isdir(rad_subdir):
                # Search for files containing "rsds" in the rad_ folder
                found_files = sorted(glob.glob(os.path.join(rad_subdir, "*rsds*.nc")))
                if found_files:
                    nc_files.extend(found_files)
                    print(f"  Found files in subdirectory: {os.path.basename(rad_subdir)}/")
                    break
                # Fallback: if no rsds files found, try all .nc files
                if len(nc_files) == 0:
                    found_files = sorted(glob.glob(os.path.join(rad_subdir, "*.nc")))
                    if found_files:
                        nc_files.extend(found_files)
                        print(f"  Found files in subdirectory: {os.path.basename(rad_subdir)}/")
                        break
    
    # Pattern 3: Check subdirectory named exactly after the variable
    if len(nc_files) == 0:
        var_dir = os.path.join(netcdf_dir, variable)
        if os.path.exists(var_dir) and os.path.isdir(var_dir):
            nc_files = sorted(glob.glob(os.path.join(var_dir, "*.nc")))
            if len(nc_files) > 0:
                print(f"  Found files in subdirectory: {variable}/")
    
    if len(nc_files) == 0:
        print(f"  ERROR: No NetCDF files found in {netcdf_dir}")
        print(f"  Searched patterns:")
        print(f"    - {netcdf_dir}/*{variable}*.nc")
        print(f"    - {netcdf_dir}/{variable}_*/*.nc")
        if variable == 'rsds':
            print(f"    - {netcdf_dir}/rad_*/*rsds*.nc")
            print(f"    - {netcdf_dir}/rad_*/*.nc")
        print(f"    - {netcdf_dir}/{variable}/*.nc")
        return None
    
    print(f"  Found {len(nc_files)} NetCDF files")
    
    # Cache coordinate information from first file
    lat_name = None
    lon_name = None
    time_name = None
    lat_idx = None
    lon_idx = None
    actual_lat = None
    actual_lon = None
    var_name = None
    
    # List to store daily data
    all_data = []
    
    # Process first file to get coordinate structure
    if len(nc_files) > 0:
        try:
            ds_sample = xr.open_dataset(nc_files[0], decode_times=False)
            
            # Get variable name
            for v in ds_sample.data_vars:
                if variable in v.lower() or v.lower() in variable.lower():
                    var_name = v
                    break
            
            # For rsds, also check for "rad" variable name (some datasets use "rad" instead of "rsds")
            if var_name is None and variable == 'rsds':
                for v in ds_sample.data_vars:
                    if 'rad' in v.lower() and 'rsds' not in v.lower():
                        var_name = v
                        break
            
            if var_name is None:
                possible_names = [variable, variable.upper(), f'{variable}_day']
                # For rsds, also try "rad" as variable name
                if variable == 'rsds':
                    possible_names.extend(['rad', 'RAD', 'rad_day'])
                for name in possible_names:
                    if name in ds_sample.data_vars:
                        var_name = name
                        break
            
            # Get coordinate names
            for coord in ds_sample.coords:
                coord_lower = coord.lower()
                if 'lat' in coord_lower:
                    lat_name = coord
                elif 'lon' in coord_lower:
                    lon_name = coord
                elif 'time' in coord_lower:
                    time_name = coord
            
            if lat_name and lon_name:
                # Check coordinate bounds of the NetCDF file
                lat_values = ds_sample[lat_name].values
                lon_values = ds_sample[lon_name].values
                
                lat_min = float(np.min(lat_values))
                lat_max = float(np.max(lat_values))
                lon_min = float(np.min(lon_values))
                lon_max = float(np.max(lon_values))
                
                # Check if target coordinate is within file bounds
                lat_in_bounds = lat_min <= target_lat <= lat_max
                lon_in_bounds = lon_min <= target_lon <= lon_max
                
                print(f"  Grid bounds: Lat [{lat_min:.4f}, {lat_max:.4f}], Lon [{lon_min:.4f}, {lon_max:.4f}]")
                print(f"  Target coordinate: ({target_lat:.4f}, {target_lon:.4f})")
                
                if not lat_in_bounds:
                    print(f"  [WARNING] Target latitude {target_lat:.4f} is OUTSIDE file bounds [{lat_min:.4f}, {lat_max:.4f}]")
                    print(f"  [WARNING] This may cause all values to be zero or incorrect!")
                if not lon_in_bounds:
                    print(f"  [WARNING] Target longitude {target_lon:.4f} is OUTSIDE file bounds [{lon_min:.4f}, {lon_max:.4f}]")
                    print(f"  [WARNING] This may cause all values to be zero or incorrect!")
                
                if lat_in_bounds and lon_in_bounds:
                    print(f"  [OK] Target coordinate is within file bounds")
                
                # Find nearest grid point (cache indices)
                lat_idx = np.abs(lat_values - target_lat).argmin()
                lon_idx = np.abs(lon_values - target_lon).argmin()
                
                actual_lat = float(lat_values[lat_idx])
                actual_lon = float(lon_values[lon_idx])
                
                # Check if within tolerance
                lat_diff = abs(actual_lat - target_lat)
                lon_diff = abs(actual_lon - target_lon)
                
                if lat_diff > tolerance or lon_diff > tolerance:
                    print(f"  [WARNING] Nearest point ({actual_lat:.4f}, {actual_lon:.4f}) is outside tolerance")
                    print(f"  [WARNING] Distance: {lat_diff:.4f}° lat, {lon_diff:.4f}° lon (tolerance: {tolerance:.4f}°)")
                else:
                    print(f"  [OK] Using grid point: ({actual_lat:.4f}, {actual_lon:.4f})")
                    print(f"  [OK] Distance from target: {lat_diff:.4f}° lat, {lon_diff:.4f}° lon")
            
            ds_sample.close()
            
        except Exception as e:
            print(f"  Warning: Could not read sample file: {e}")
    
    if var_name is None or lat_idx is None or lon_idx is None:
        print(f"  ERROR: Could not determine coordinate structure")
        return None
    
    # Process all files with progress bar
    print(f"  Processing files...")
    for nc_file in tqdm(nc_files, desc=f"  {variable}", unit="file"):
        try:
            # Open NetCDF file with minimal decoding for speed
            ds = xr.open_dataset(nc_file, decode_times=False)
            
            # Extract data using cached indices
            data = ds[var_name].isel({lat_name: lat_idx, lon_name: lon_idx})
            
            # Convert to numpy array (load into memory)
            values = data.values
            if values.ndim > 1:
                values = values.flatten()
            
            # Get time values - try multiple methods to ensure accuracy and handle leap years (366 days)
            time_values = None
            
            # Method 1: Try to use time coordinate from NetCDF file (most reliable)
            if time_name and time_name in ds.coords:
                try:
                    time_coord = ds[time_name]
                    if len(time_coord) == len(values):
                        # Try to decode times
                        try:
                            # Decode time coordinate
                            time_decoded = xr.decode_cf(ds[[time_name]])[time_name]
                            time_values = pd.to_datetime(time_decoded.values)
                            if len(time_values) == len(values):
                                pass  # Success - using decoded time coordinate
                        except:
                            # If decoding fails, try manual conversion
                            if hasattr(time_coord, 'units') and 'days since' in time_coord.units.lower():
                                base_date_str = time_coord.units.split('since')[1].strip().split()[0]
                                base_date = pd.to_datetime(base_date_str)
                                time_values = base_date + pd.to_timedelta(time_coord.values, unit='D')
                                if len(time_values) != len(values):
                                    time_values = None
                except Exception as e:
                    pass  # Fall back to other methods
            
            # Method 2: Extract year from filename and create date range
            # This method automatically handles leap years (366 days) correctly
            if time_values is None:
                year = None
                filename = os.path.basename(nc_file)
                all_years = re.findall(r'\d{4}', filename)
                for year_str in all_years:
                    year_candidate = int(year_str)
                    if 2000 <= year_candidate <= 2100:
                        year = year_candidate
                        break
                
                if year:
                    # Create dates based on ACTUAL data length
                    # pd.date_range with freq='D' automatically handles leap years
                    # For leap years (e.g., 2024, 2028), it will include Feb 29 (366 days)
                    # For non-leap years, it will have 365 days
                    time_values = pd.date_range(start=f'{year}-01-01', periods=len(values), freq='D')
                else:
                    # Fallback: use 2035 as default (start of typical CMIP6 data range)
                    time_values = pd.date_range(start='2035-01-01', periods=len(values), freq='D')
            
            # Ensure we have the correct number of dates matching the data
            # This handles edge cases where time coordinate might not match exactly
            if len(time_values) != len(values):
                if len(time_values) > len(values):
                    time_values = time_values[:len(values)]
                else:
                    # Extend if needed (shouldn't happen normally, but handle it)
                    additional_days = len(values) - len(time_values)
                    last_date = time_values[-1]
                    additional_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=additional_days, freq='D')
                    time_values = pd.concat([pd.Series(time_values), pd.Series(additional_dates)]).values
            
            # Create DataFrame for this file
            # Use actual data length to ensure all days are included (365 or 366 for leap years)
            if len(values) > 0:
                df_file = pd.DataFrame({
                    'date': time_values[:len(values)],
                    'value': values
                })
                all_data.append(df_file)
            
            ds.close()
            
        except Exception as e:
            tqdm.write(f"    Error processing {os.path.basename(nc_file)}: {e}")
            continue
    
    if len(all_data) == 0:
        print(f"  ERROR: No data extracted")
        return None
    
    # Combine all data
    print(f"  Combining data from {len(all_data)} files...")
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Sort by date
    combined_df = combined_df.sort_values('date').reset_index(drop=True)
    
    # Remove duplicate dates (keep first occurrence)
    combined_df = combined_df.drop_duplicates(subset='date', keep='first')
    
    elapsed_time = time.time() - start_time
    print(f"  ✓ Extracted {len(combined_df):,} daily records in {elapsed_time:.1f} seconds")
    print(f"  Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")
    
    return combined_df

## Section 4: MET Conversion Functions

In [5]:
def calculate_tav_amp(df):
    """
    Calculate annual average temperature (tav) and annual amplitude (amp).
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with 'date' as index and 'maxt' and 'mint' columns
    
    Returns:
    --------
    tuple: (tav, amp)
        tav: Annual average ambient temperature
        amp: Annual amplitude in mean monthly temperature
    """
    # Calculate daily mean temperature
    df = df.copy()
    df['tmean'] = (df['maxt'] + df['mint']) / 2.0
    
    # Calculate monthly means
    df['year'] = df.index.year
    df['month'] = df.index.month
    monthly_means = df.groupby(['year', 'month'])['tmean'].mean()
    
    # Calculate overall annual average (tav)
    tav = df['tmean'].mean()
    
    # Calculate annual amplitude (amp)
    # Average of all January means minus average of all July means, divided by 2
    jan_means = monthly_means[monthly_means.index.get_level_values('month') == 1].mean()
    jul_means = monthly_means[monthly_means.index.get_level_values('month') == 7].mean()
    amp = (jan_means - jul_means) / 2.0
    
    return tav, amp

In [6]:
def calculate_saturation_vapor_pressure(temperature):
    """
    Calculate saturation vapor pressure (kPa) at a given temperature using SILO method.
    
    Parameters:
    -----------
    temperature : float or array
        Temperature in °C
    
    Returns:
    --------
    float or array
        Saturation vapor pressure in kPa
    """
    # SILO formula: e_s(T) = 0.611 × exp(17.27 × T / (T + 237.3))
    return 0.611 * np.exp(17.27 * temperature / (temperature + 237.3))


def calculate_vapor_pressure(hurs_df, tasmax_df, tasmin_df):
    """
    Calculate vapor pressure (hPa) from mean relative humidity and temperature using SILO method.
    
    Parameters:
    -----------
    hurs_df : pd.DataFrame
        DataFrame with date and value (mean relative humidity %) columns
    tasmax_df : pd.DataFrame
        DataFrame with date and value (maximum temperature °C) columns
    tasmin_df : pd.DataFrame
        DataFrame with date and value (minimum temperature °C) columns
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with date and value (vapor pressure hPa) columns
    """
    # Merge temperature dataframes
    temp_df = tasmax_df.merge(tasmin_df, on='date', suffixes=('_max', '_min'))
    temp_df['tmean'] = (temp_df['value_max'] + temp_df['value_min']) / 2.0
    
    # Merge with mean humidity
    merged = hurs_df.merge(temp_df[['date', 'tmean']], on='date')
    
    # Calculate saturation vapor pressure at mean temperature (in kPa)
    # SILO formula: e_s(T) = 0.611 × exp(17.27 × T / (T + 237.3))
    merged['es_kpa'] = calculate_saturation_vapor_pressure(merged['tmean'])
    
    # Calculate actual vapor pressure using mean relative humidity (in kPa)
    # e_a = (hurs/100) × e_s(T_mean)
    merged['ea_kpa'] = (merged['value'] / 100.0) * merged['es_kpa']
    
    # Convert to SILO VP units (hPa): VP(hPa) = 10 × e_a(kPa)
    merged['vp'] = 10.0 * merged['ea_kpa']
    
    # Return DataFrame with date and vp columns
    vp_df = merged[['date', 'vp']].copy()
    vp_df = vp_df.rename(columns={'vp': 'value'})
    
    return vp_df

In [7]:
def create_met_file(tasmax_df, tasmin_df, pr_df, rsds_df, hurs_df=None, scenario=None, 
                    output_dir=None, latitude=None, longitude=None, model=None, 
                    start_year=None, end_year=None):
    """
    Create MET format file from tasmax, tasmin, pr, rsds, and optional hurs DataFrames.
    
    Parameters:
    -----------
    tasmax_df : pd.DataFrame
        DataFrame with date and value columns for maximum temperature
    tasmin_df : pd.DataFrame
        DataFrame with date and value columns for minimum temperature
    pr_df : pd.DataFrame
        DataFrame with date and value columns for precipitation
    rsds_df : pd.DataFrame
        DataFrame with date and value columns for surface downwelling shortwave radiation (W/m²) - REQUIRED
    hurs_df : pd.DataFrame, optional
        DataFrame with date and value columns for relative humidity (%) - if provided, VP will be calculated
    scenario : str
        Scenario name (e.g., SSP585 or SSP245)
    output_dir : str
        Output directory path
    latitude : float
        Latitude in decimal degrees
    longitude : float
        Longitude in decimal degrees
    model : str
        Model name (e.g., "ACCESS CM2")
    start_year : int, optional
        Start year for MET file. Only data from this year onwards will be included.
        If None, uses the minimum date from the data.
    end_year : int, optional
        End year for MET file. Only data up to this year will be included.
        If None, uses the maximum date from the data.
    
    Returns:
    --------
    tuple: (tav, amp, num_rows, final_date_range)
        tav: Annual average ambient temperature
        amp: Annual amplitude in mean monthly temperature
        num_rows: Number of rows in the MET file
        final_date_range: Dict with 'start' and 'end' dates from the final data
    """
    # Merge all dataframes on date
    merged = tasmax_df.copy()
    merged = merged.rename(columns={'value': 'maxt'})
    merged['date'] = pd.to_datetime(merged['date'])
    
    # Merge tasmin
    tasmin_df['date'] = pd.to_datetime(tasmin_df['date'])
    merged = merged.merge(tasmin_df[['date', 'value']], on='date', how='outer')
    merged = merged.rename(columns={'value': 'mint'})
    
    # Merge pr (precipitation/rain)
    pr_df['date'] = pd.to_datetime(pr_df['date'])
    merged = merged.merge(pr_df[['date', 'value']], on='date', how='outer')
    merged = merged.rename(columns={'value': 'rain'})
    
    # Merge rsds (radiation) - REQUIRED
    # rsds is in W/m², convert to MJ/m² by multiplying by 0.0864 (seconds per day / 1e6)
    if rsds_df is None or len(rsds_df) == 0:
        raise ValueError("rsds_df is required but is None or empty")
    
    rsds_df['date'] = pd.to_datetime(rsds_df['date'])
    # Convert W/m² to MJ/m² (multiply by seconds per day / 1e6)
    rsds_df['value_mj'] = rsds_df['value'] * 0.0864
    merged = merged.merge(rsds_df[['date', 'value_mj']], on='date', how='outer')
    merged = merged.rename(columns={'value_mj': 'radn'})
    
    # Calculate vp (vapor pressure) if hurs is provided, otherwise leave blank
    if hurs_df is not None and len(hurs_df) > 0:
        # Calculate VP using SILO method
        vp_df = calculate_vapor_pressure(hurs_df, tasmax_df, tasmin_df)
        vp_df['date'] = pd.to_datetime(vp_df['date'])
        merged = merged.merge(vp_df[['date', 'value']], on='date', how='left')  # Use 'left' to keep only dates that exist in merged
        merged = merged.rename(columns={'value': 'vp'})
        # Ensure VP is numeric only where it was calculated, leave NaN where hurs data was missing
        merged['vp'] = pd.to_numeric(merged['vp'], errors='coerce')
        # Count how many VP values were calculated vs missing
        vp_calculated = merged['vp'].notna().sum()
        vp_missing = merged['vp'].isna().sum()
        print(f"  [OK] Calculated vapor pressure for {vp_calculated} days")
        if vp_missing > 0:
            print(f"  [WARNING] VP data missing for {vp_missing} days (hurs data not available) - these will be left blank")
    else:
        # vp (vapor pressure) is left blank if hurs not available
        merged['vp'] = ''
        print(f"  [INFO] hurs not provided - VP left blank")
    
    # Sort by date
    merged = merged.sort_values('date').reset_index(drop=True)
    
    # Filter data to only include dates from start_year onwards and up to end_year
    original_count_before_filter = len(merged)
    original_date_range = f"{merged['date'].min()} to {merged['date'].max()}" if len(merged) > 0 else "no data"
    
    if start_year is not None:
        start_date = pd.Timestamp(year=start_year, month=1, day=1)
        merged = merged[merged['date'] >= start_date].copy()
        removed_count = original_count_before_filter - len(merged)
        if removed_count > 0:
            print(f"  [INFO] Filtered data to start from {start_year}: removed {removed_count:,} earlier records")
            if len(merged) == 0:
                print(f"  [WARNING] All data was filtered out! Original date range was {original_date_range}")
                print(f"  [WARNING] Consider setting START_YEAR=None or to an earlier year")
    
    if end_year is not None:
        end_date = pd.Timestamp(year=end_year, month=12, day=31)
        count_before_end_filter = len(merged)
        merged = merged[merged['date'] <= end_date].copy()
        removed_count = count_before_end_filter - len(merged)
        if removed_count > 0:
            print(f"  [INFO] Filtered data to end at {end_year}: removed {removed_count:,} later records")
    
    # CRITICAL FIX: Create a complete date range to ensure all days are included
    # This ensures leap years have 366 days and non-leap years have 365 days
    # IMPORTANT: Use the actual min/max dates from the filtered data
    # Only extend to full years if the data actually covers those dates
    if len(merged) == 0:
        raise ValueError("No data remaining after filtering! Check START_YEAR and END_YEAR settings.")
    
    # Get actual min and max dates from filtered data (these are the dates that actually have data)
    actual_min_date = merged['date'].min()
    actual_max_date = merged['date'].max()
    
    # CRITICAL FIX: Normalize dates to remove time components for proper matching
    # This ensures dates like "1986-01-02 09:00:00" match "1986-01-02 00:00:00"
    # Without this, reindexing fails and all values become NaN (then filled with 0.0)
    merged['date'] = pd.to_datetime(merged['date']).dt.normalize()
    
    # Remove duplicate dates (keep first occurrence) - duplicates can occur after normalization
    # if different variables had the same date with different time components
    duplicate_count = merged.duplicated(subset='date').sum()
    if duplicate_count > 0:
        print(f"  [INFO] Found {duplicate_count} duplicate dates after normalization, removing duplicates (keeping first occurrence)")
        merged = merged.drop_duplicates(subset='date', keep='first').reset_index(drop=True)
    
    # Use the actual data range, rounded to full years
    # This ensures we don't create dates that don't exist in the source data
    min_date = pd.Timestamp(year=actual_min_date.year, month=1, day=1)
    max_date = pd.Timestamp(year=actual_max_date.year, month=12, day=31)
    
    # Create complete date range (includes all days, including day 366 for leap years)
    complete_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    complete_date_range = pd.to_datetime(complete_date_range).normalize()
    
    # Set date as index for reindexing
    merged = merged.set_index('date')
    
    # Reindex to include all days in the complete range
    merged = merged.reindex(complete_date_range)
    
    # Fill missing values for numeric columns using forward fill then backward fill
    # This handles gaps in the data gracefully
    # NOTE: VP is NOT filled - if hurs data is missing, VP remains blank
    numeric_cols = ['maxt', 'mint', 'rain', 'radn']
    for col in numeric_cols:
        if col in merged.columns:
            # Forward fill first, then backward fill to handle gaps at start/end
            # This ensures that if data starts mid-year, early dates are filled from the first available value
            merged[col] = merged[col].ffill().bfill()
            # Only fill remaining NaNs with 0 if there's absolutely no data (shouldn't happen after ffill/bfill)
            # But check if we have any valid data first
            if merged[col].notna().any():
                # If we have some data, use the mean of available values for any remaining NaNs
                # This is safer than 0.0 and handles edge cases better
                mean_val = merged[col].mean()
                if pd.notna(mean_val):
                    merged[col] = merged[col].fillna(mean_val)
                else:
                    merged[col] = merged[col].fillna(0.0)
            else:
                # No data at all - this shouldn't happen, but fill with 0 as last resort
                print(f"  [WARNING] Column {col} has no valid data! All values will be 0.0")
                merged[col] = merged[col].fillna(0.0)
    
    # Handle VP separately - DO NOT fill missing VP values
    # If hurs data was missing, VP should remain blank (NaN or empty string)
    # This includes cases where date range was extended but source data doesn't exist (e.g., Dec 31, 2064)
    if 'vp' in merged.columns:
        # Convert all NaN values to empty string (whether numeric or object type)
        merged.loc[merged['vp'].isna(), 'vp'] = ''
        # If VP was numeric and now has empty strings, convert dtype to object to allow mixed types
        if merged['vp'].dtype != 'object':
            # Check if we have any empty strings
            if (merged['vp'] == '').any():
                # Convert to object to allow both numeric and string values
                merged['vp'] = merged['vp'].astype(object)
                # Convert numeric values back to float
                mask = (merged['vp'] != '') & (merged['vp'].notna())
                merged.loc[mask, 'vp'] = pd.to_numeric(merged.loc[mask, 'vp'], errors='coerce')
                # Convert any resulting NaN back to empty string
                merged.loc[merged['vp'].isna(), 'vp'] = ''
    
    # Reset index to get date back as a column
    merged = merged.reset_index()
    merged = merged.rename(columns={'index': 'date'})
    
    # Diagnostic: Check data before calculating tav/amp
    if len(merged) > 0:
        maxt_valid = merged['maxt'].notna().sum()
        mint_valid = merged['mint'].notna().sum()
        maxt_nonzero = (merged['maxt'] != 0).sum() if 'maxt' in merged.columns else 0
        mint_nonzero = (merged['mint'] != 0).sum() if 'mint' in merged.columns else 0
        print(f"  [INFO] After date range creation: {len(merged)} total rows")
        print(f"  [INFO] Valid maxt: {maxt_valid}, non-zero maxt: {maxt_nonzero}")
        print(f"  [INFO] Valid mint: {mint_valid}, non-zero mint: {mint_nonzero}")
        if maxt_nonzero == 0 or mint_nonzero == 0:
            print(f"  [WARNING] All temperature values are zero! This may indicate a data issue.")
            print(f"  [WARNING] Date range: {merged['date'].min()} to {merged['date'].max()}")
    
    # Calculate tav and amp (use only non-NaN values for calculation)
    merged_temp = merged[['date', 'maxt', 'mint']].copy()
    merged_temp = merged_temp.set_index('date')
    merged_temp.index = pd.to_datetime(merged_temp.index)
    tav, amp = calculate_tav_amp(merged_temp)
    
    # Create year and day columns
    merged['year'] = merged['date'].dt.year
    merged['day'] = merged['date'].dt.dayofyear
    
    # Add empty columns for evap and code
    merged['evap'] = ''  # Leave blank
    merged['code'] = '222222'  # Hardcoded code value for all rows
    
    # Ensure vp column exists (should already be set above)
    if 'vp' not in merged.columns:
        merged['vp'] = ''
    
    # Convert VP to appropriate format for output
    # - Numeric values (calculated) should remain as float
    # - Missing values (NaN or empty string) should be empty string
    if merged['vp'].dtype != 'object':
        # VP is numeric - convert NaN to empty string, keep valid values as float
        merged.loc[merged['vp'].isna(), 'vp'] = ''
        # Convert non-empty values to float
        mask = (merged['vp'] != '') & (merged['vp'].notna())
        merged.loc[mask, 'vp'] = pd.to_numeric(merged.loc[mask, 'vp'], errors='coerce')
    # If VP is already string (empty), keep it as is
    
    # Check for blank VP values and issue warning
    vp_blank_count = ((merged['vp'] == '') | (merged['vp'].isna())).sum()
    if vp_blank_count > 0:
        # Find which years have blank VP values
        blank_vp_dates = merged[((merged['vp'] == '') | (merged['vp'].isna())) & merged['date'].notna()]
        if len(blank_vp_dates) > 0:
            blank_years = sorted(blank_vp_dates['date'].dt.year.unique())
            years_str = ', '.join(map(str, blank_years))
            print(f"  [WARNING] Found {vp_blank_count} days with blank VP values (missing hurs data)")
            print(f"  [WARNING] Years affected: {years_str}")
            print(f"  [WARNING] These VP values will appear as blank spaces in the output files")
    
    met_data = merged[['year', 'day', 'radn', 'maxt', 'mint', 'rain', 'evap', 'vp', 'code']].copy()
    
    # Prepare header
    current_date = datetime.now().strftime('%Y%m%d')
    model_scenario = f"{model.replace(' ', '_')}_{scenario}" if model and scenario else "CMIP6"
    
    header = f"""[weather.met.weather]
!Your Ref:  "
latitude = {latitude:.2f}  (DECIMAL DEGREES)
longitude =  {longitude:.2f}  (DECIMAL DEGREES)
tav = {tav:.2f} (oC) ! Annual average ambient temperature.
amp = {amp:.2f} (oC) ! Annual amplitude in mean monthly temperature.
!Data Extracted from CMIP6 {model} {scenario} dataset on {current_date} for APSIM
!As evaporation is read at 9am, it has been shifted to day before
!ie The evaporation measured on 20 April is in row for 19 April
!The 6 digit code indicates the source of the 6 data columns
!0 actual observation, 1 actual observation composite station
!2 interpolated from daily observations
!3 interpolated from daily observations using anomaly interpolation method for CLIMARC data
!6 synthetic pan
!7 interpolated long term averages
!more detailed two digit codes are available in SILO's 'Standard' format files
!
!For further information see the documentation on the datadrill
!  http://www.longpaddock.qld.gov.au/silo
!
year  day radn  maxt   mint  rain  evap    vp   code
 ()   () (MJ/m^2) (oC)  (oC)  (mm)  (mm) (hPa)     ()
"""
    
    # Create output filename with coordinate-based naming
    # Format: {Model}_{Scenario}_{Lat}_{Lon}.met (e.g., ACCESS_CM2_SSP245_-31.75_117.60.met)
    lat_str = f"{latitude:.2f}"
    lon_str = f"{longitude:.2f}"
    output_filename = f"{model_scenario}_{lat_str}_{lon_str}.met"
    output_path = os.path.join(output_dir, output_filename)
    
    # Write MET file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(header)
        # Write data rows
        for _, row in met_data.iterrows():
            # Format the row with proper spacing
                        # Format numbers with proper spacing
            radn_val = row['radn'] if row['radn'] != '' and pd.notna(row['radn']) else ''
            evap_val = row['evap'] if row['evap'] != '' and pd.notna(row['evap']) else ''
            
            if radn_val != '':
                radn_str = f"{float(radn_val):6.1f}"
            else:
                radn_str = "      "  # 6 spaces
                
            if evap_val != '':
                evap_str = f"{float(evap_val):6.1f}"
            else:
                evap_str = "      "  # 6 spaces
                
            # Handle VP - can be numeric (calculated) or empty string (blank)
            if pd.notna(row['vp']) and row['vp'] != '':
                try:
                    vp_str = f"{float(row['vp']):6.1f}"
                except (ValueError, TypeError):
                    vp_str = "      "  # 6 spaces
            else:
                vp_str = "      "  # 6 spaces
            
                        # Code is hardcoded to '222222' for all rows
            code_str = "222222"
            
            # Handle NaN values for maxt, mint, rain - use 0.0 as default
            maxt_val = row['maxt'] if pd.notna(row['maxt']) else 0.0
            mint_val = row['mint'] if pd.notna(row['mint']) else 0.0
            rain_val = row['rain'] if pd.notna(row['rain']) else 0.0
            
            # Format with proper column widths
            line = f"{int(row['year']):4d} {int(row['day']):4d} {radn_str} {maxt_val:6.1f} {mint_val:6.1f} {rain_val:6.1f} {evap_str} {vp_str} {code_str}\n"
            f.write(line)
    
    # Count blank VP values in final output for warning (after file is written)
    final_blank_vp = ((met_data['vp'] == '') | (met_data['vp'].isna())).sum()
    
    print(f"  [OK] Created MET file: {output_filename}")
    
    # Final warning about blank VP values
    if final_blank_vp > 0:
        blank_years_list = sorted(met_data[((met_data['vp'] == '') | (met_data['vp'].isna()))]['year'].unique())
        years_str = ', '.join(map(str, blank_years_list))
        print(f"  [WARNING] ========================================")
        print(f"  [WARNING] BLANK VP VALUES DETECTED!")
        print(f"  [WARNING] {final_blank_vp} days have blank VP values (missing hurs data)")
        print(f"  [WARNING] Affected years: {years_str}")
        print(f"  [WARNING] These will appear as blank spaces in MET/CSV files")
        print(f"  [WARNING] ========================================")
    
    # Also create CSV version with same structure
    csv_filename = f"{model_scenario}_{lat_str}_{lon_str}.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    
    # Write CSV (without header comments, just data)
    met_data.to_csv(csv_path, index=False, encoding='utf-8', float_format='%.1f')
    print(f"  [OK] Created CSV file: {csv_filename}")
    
    # Get date range from merged dataframe (date column exists after reset_index)
    if 'date' in merged.columns and len(merged) > 0:
        final_date_range = {
            'start': merged['date'].min(),
            'end': merged['date'].max()
        }
    else:
        # Fallback: reconstruct from year and day if needed
        final_date_range = {'start': None, 'end': None}
    
    return tav, amp, len(met_data), final_date_range

## Section 5: Main Processing

This section extracts data for all variables and creates the MET file.

In [8]:
def process_coordinate(model, scenario, latitude, longitude, variables, cmip6_base_dir, output_dir, tolerance=0.01):
    """
    Main processing function for user-provided coordinate.
    Extract all variables from NC files and convert to MET format.
    
    Parameters:
    -----------
    model : str
        Model name (e.g., "ACCESS CM2")
    scenario : str
        Scenario name (e.g., "SSP245")
    latitude : float
        Target latitude
    longitude : float
        Target longitude
    variables : list
        List of variable names to extract
    cmip6_base_dir : str
        Base directory containing Model Scenario folders
    output_dir : str
        Output directory for results
    tolerance : float
        Coordinate matching tolerance
    
    Returns:
    --------
    dict: Summary statistics
    """
    print("="*70)
    print(f"Processing Coordinate: ({latitude:.6f}, {longitude:.6f})")
    print(f"Model: {model}, Scenario: {scenario}")
    print("="*70)
    
    # Construct data directory path
    data_dir = os.path.join(cmip6_base_dir, f"{model} {scenario}")
    
    if not os.path.exists(data_dir):
        print(f"ERROR: Data directory not found: {data_dir}")
        return None
    
    print(f"\nData directory: {data_dir}")
    
    # Extract data for all variables
    extracted_data = {}
    
    for variable in variables:
        print(f"\n{'='*70}")
        print(f"Processing variable: {variable}")
        print(f"{'='*70}")
        
        # Extract data from NetCDF files
        df = extract_daily_data_from_netcdf(
            data_dir, 
            variable, 
            latitude, 
            longitude, 
            tolerance=tolerance
        )
        
        if df is not None and len(df) > 0:
            extracted_data[variable] = df
            
            # Save individual variable CSV
            # Format: {Model}_{Scenario}_{Lat}_{Lon}_{variable}.csv
            lat_str = f"{latitude:.2f}"
            lon_str = f"{longitude:.2f}"
            model_scenario = f"{model.replace(' ', '_')}_{scenario}"
            csv_filename = f"{model_scenario}_{lat_str}_{lon_str}_{variable}.csv"
            csv_path = os.path.join(output_dir, csv_filename)
            df.to_csv(csv_path, index=False, encoding='utf-8', float_format='%.6f')
            print(f"  [OK] Saved CSV: {csv_filename}")
        else:
            print(f"  WARNING: No data extracted for {variable}")
    
    # Check if required variables are available for MET conversion
    # Note: rsds is now mandatory (required for radn in MET format)
    required_vars = ['tasmax', 'tasmin', 'pr', 'rsds']
    missing_vars = [v for v in required_vars if v not in extracted_data]
    
    if missing_vars:
        print(f"\nERROR: Missing required variables for MET conversion: {missing_vars}")
        return None
    
    # Create MET file
    print(f"\n{'='*70}")
    print("Creating MET file...")
    print(f"{'='*70}")
    
    # Get required variables
    tasmax_df = extracted_data['tasmax']
    tasmin_df = extracted_data['tasmin']
    pr_df = extracted_data['pr']
    
    # Get rsds variable for MET format (now mandatory)
    rsds_df = extracted_data.get('rsds', None)
    if rsds_df is None:
        print(f"  ERROR: rsds is required but was not extracted")
        return None
    
    # Get hurs variable for VP calculation (optional but recommended)
    hurs_df = extracted_data.get('hurs', None)
    if hurs_df is not None:
        print(f"  [INFO] hurs data available - VP will be calculated using SILO method")
    else:
        print(f"  [INFO] hurs data not available - VP will be left blank")
    
    # Note: code is hardcoded to '222222'
    
    tav, amp, num_rows, final_date_range = create_met_file(
        tasmax_df=tasmax_df,
        tasmin_df=tasmin_df,
        pr_df=pr_df,
        rsds_df=rsds_df,
        hurs_df=hurs_df,
        scenario=scenario,
        output_dir=output_dir,
        latitude=latitude,
        longitude=longitude,
        model=model,
        start_year=START_YEAR,
        end_year=END_YEAR
    )
    
    # Summary
    summary = {
        'latitude': latitude,
        'longitude': longitude,
        'model': model,
        'scenario': scenario,
        'variables_extracted': list(extracted_data.keys()),
        'num_variables': len(extracted_data),
        'tav': tav,
        'amp': amp,
        'num_rows': num_rows,
        'date_range': final_date_range if final_date_range['start'] is not None else {
            'start': tasmax_df['date'].min(),
            'end': tasmax_df['date'].max()
        }
    }
    
    print(f"\n{'='*70}")
    print("Processing Summary")
    print(f"{'='*70}")
    print(f"  Variables extracted: {len(extracted_data)}")
    print(f"    - {', '.join(extracted_data.keys())}")
    print(f"  MET file rows: {num_rows}")
    print(f"  Date range: {summary['date_range']['start']} to {summary['date_range']['end']}")
    print(f"  tav (annual average temp): {tav:.2f} °C")
    print(f"  amp (annual amplitude): {amp:.2f} °C")
    print(f"  Output directory: {output_dir}")
    print(f"{'='*70}")
    
    return summary

## Configuration Summary

Review the configuration below before running the main processing.

In [9]:
# Display full configuration
print("="*70)
print("CONFIGURATION SUMMARY")
print("="*70)
print(f"\nModel: {MODEL}")
print(f"Scenario: {SCENARIO}")
print(f"\nCoordinates:")
print(f"  Latitude: {LATITUDE:.6f}°")
print(f"  Longitude: {LONGITUDE:.6f}°")
print(f"  Tolerance: {COORD_TOLERANCE} degrees (≈ {COORD_TOLERANCE * 111:.1f} km)")
print(f"\nVariables to process ({len(VARIABLES)}):")
for var in VARIABLES:
    if var == 'tasmax':
        print(f"  - {var} → maxt (maximum temperature)")
    elif var == 'tasmin':
        print(f"  - {var} → mint (minimum temperature)")
    elif var == 'pr':
        print(f"  - {var} → rain (precipitation)")
    elif var == 'rsds':
        print(f"  - {var} → radn (radiation, W/m² → MJ/m²)")
print(f"\nDirectories:")
print(f"  CMIP6 Base: {CMIP6_BASE_DIR}")
print(f"  Data Directory: {os.path.join(CMIP6_BASE_DIR, f'{MODEL} {SCENARIO}')}")
print(f"  Output Directory: {OUTPUT_DIR}")
print(f"\nOutput Files:")
print(f"  MET file: {MODEL.replace(' ', '_')}_{SCENARIO}_{LATITUDE:.2f}_{LONGITUDE:.2f}.met")
print(f"  CSV file: {MODEL.replace(' ', '_')}_{SCENARIO}_{LATITUDE:.2f}_{LONGITUDE:.2f}.csv")
print(f"  Variable CSVs: {len(VARIABLES)} files (one per variable)")
print("="*70)

CONFIGURATION SUMMARY

Model: ACCESS CM2
Scenario: obs

Coordinates:
  Latitude: -31.450000°
  Longitude: 117.550000°
  Tolerance: 0.01 degrees (≈ 1.1 km)

Variables to process (5):
  - tasmax → maxt (maximum temperature)
  - tasmin → mint (minimum temperature)
  - pr → rain (precipitation)
  - rsds → radn (radiation, W/m² → MJ/m²)

Directories:
  CMIP6 Base: C:\Users\ibian\Desktop\ClimAdapt\CMIP6
  Data Directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 obs
  Output Directory: C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.45_117.55

Output Files:
  MET file: ACCESS_CM2_obs_-31.45_117.55.met
  CSV file: ACCESS_CM2_obs_-31.45_117.55.csv
  Variable CSVs: 5 files (one per variable)


In [10]:
# Execute main processing
print("\n" + "="*70)
print("STARTING PROCESSING")
print("="*70)
print(f"Model: {MODEL}")
print(f"Scenario: {SCENARIO}")
print(f"Coordinates: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"Variables to process: {len(VARIABLES)} ({', '.join(VARIABLES)})")
print(f"Output directory: {OUTPUT_DIR}")
print("="*70 + "\n")

summary = process_coordinate(
    model=MODEL,
    scenario=SCENARIO,
    latitude=LATITUDE,
    longitude=LONGITUDE,
    variables=VARIABLES,
    cmip6_base_dir=CMIP6_BASE_DIR,
    output_dir=OUTPUT_DIR,
    tolerance=COORD_TOLERANCE
)

if summary:
    print("\n" + "="*70)
    print("✓ PROCESSING COMPLETED SUCCESSFULLY!")
    print("="*70)
    print(f"\nOutput files created:")
    print(f"  - MET file: {MODEL.replace(' ', '_')}_{SCENARIO}_{LATITUDE:.2f}_{LONGITUDE:.2f}.met")
    print(f"  - CSV file: {MODEL.replace(' ', '_')}_{SCENARIO}_{LATITUDE:.2f}_{LONGITUDE:.2f}.csv")
    print(f"  - Individual variable CSVs: {len(summary['variables_extracted'])} files")
    print(f"\nAll files saved to: {OUTPUT_DIR}")
else:
    print("\n" + "="*70)
    print("✗ PROCESSING FAILED")
    print("="*70)
    print("Please check error messages above.")


STARTING PROCESSING
Model: ACCESS CM2
Scenario: obs
Coordinates: (-31.450000, 117.550000)
Variables to process: 5 (tasmax, tasmin, pr, rsds, hurs)
Output directory: C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.45_117.55

Processing Coordinate: (-31.450000, 117.550000)
Model: ACCESS CM2, Scenario: obs

Data directory: C:\Users\ibian\Desktop\ClimAdapt\CMIP6\ACCESS CM2 obs

Processing variable: tasmax
  Found files in subdirectory: tasmax_ACCESS CM2_obs/
  Found 30 NetCDF files
  Grid bounds: Lat [-44.5000, -10.0000], Lon [112.0000, 156.2500]
  Target coordinate: (-31.4500, 117.5500)
  [OK] Target coordinate is within file bounds
  [OK] Using grid point: (-31.4500, 117.5500)
  [OK] Distance from target: 0.0000° lat, 0.0000° lon
  Processing files...


  tasmax: 100%|██████████| 30/30 [01:15<00:00,  2.52s/file]


  Combining data from 30 files...
  ✓ Extracted 10,957 daily records in 75.9 seconds
  Date range: 1985-01-01 09:00:00 to 2014-12-31 09:00:00
  [OK] Saved CSV: ACCESS_CM2_obs_-31.45_117.55_tasmax.csv

Processing variable: tasmin
  Found files in subdirectory: tasmin_ACCESS CM2_obs/
  Found 30 NetCDF files
  Grid bounds: Lat [-44.5000, -10.0000], Lon [112.0000, 156.2500]
  Target coordinate: (-31.4500, 117.5500)
  [OK] Target coordinate is within file bounds
  [OK] Using grid point: (-31.4500, 117.5500)
  [OK] Distance from target: 0.0000° lat, 0.0000° lon
  Processing files...


  tasmin: 100%|██████████| 30/30 [01:54<00:00,  3.81s/file]


  Combining data from 30 files...
  ✓ Extracted 10,957 daily records in 114.3 seconds
  Date range: 1985-01-01 09:00:00 to 2014-12-31 09:00:00
  [OK] Saved CSV: ACCESS_CM2_obs_-31.45_117.55_tasmin.csv

Processing variable: pr
  Found files in subdirectory: pr_ACCESS CM2_obs/
  Found 30 NetCDF files
  Grid bounds: Lat [-44.5000, -10.0000], Lon [112.0000, 156.2500]
  Target coordinate: (-31.4500, 117.5500)
  [OK] Target coordinate is within file bounds
  [OK] Using grid point: (-31.4500, 117.5500)
  [OK] Distance from target: 0.0000° lat, 0.0000° lon
  Processing files...


...

  rsds: 100%|██████████| 30/30 [00:12<00:00,  2.48file/s]


  Combining data from 30 files...
  ✓ Extracted 10,957 daily records in 12.1 seconds
  Date range: 1985-01-01 12:00:00 to 2014-12-31 12:00:00
  [OK] Saved CSV: ACCESS_CM2_obs_-31.45_117.55_rsds.csv

Processing variable: hurs
  Found files in subdirectory: hurs_BARRA_obs/
  Found 30 NetCDF files
  Grid bounds: Lat [-44.5000, -10.0000], Lon [112.0000, 156.2500]
  Target coordinate: (-31.4500, 117.5500)
  [OK] Target coordinate is within file bounds
  [OK] Using grid point: (-31.4500, 117.5500)
  [OK] Distance from target: 0.0000° lat, 0.0000° lon
  Processing files...


  hurs: 100%|██████████| 30/30 [00:08<00:00,  3.40file/s]


  Combining data from 30 files...
  ✓ Extracted 10,957 daily records in 8.9 seconds
  Date range: 1985-01-01 12:00:00 to 2014-12-31 12:00:00
  [OK] Saved CSV: ACCESS_CM2_obs_-31.45_117.55_hurs.csv

Creating MET file...
  [INFO] hurs data available - VP will be calculated using SILO method
  [OK] Calculated vapor pressure for 0 days
  [WARNING] VP data missing for 21914 days (hurs data not available) - these will be left blank
  [INFO] Filtered data to start from 1986: removed 730 earlier records
  [INFO] Filtered data to end at 2014: removed 2 later records


ValueError: cannot reindex on an axis with duplicate labels

## Section 6: Documentation

### CMIP6 Data Structure and Conventions

CMIP6 (Coupled Model Intercomparison Project Phase 6) climate data is organized by:
- **Model**: Climate model name (e.g., "ACCESS CM2")
- **Scenario**: Shared Socioeconomic Pathway (e.g., "SSP245", "SSP585")
- **Variable**: Climate variable name (e.g., "tasmax", "pr")

Data files are stored in NetCDF format (.nc) and organized in folders:
`C:\Users\ibian\Desktop\ClimAdapt\CMIP6\{Model} {Scenario}\{variable}_*\*.nc`

### Coordinate System

Coordinates are provided in decimal degrees:
- **Latitude**: -90 to 90 (negative for Southern Hemisphere)
- **Longitude**: -180 to 180 (negative for Western Hemisphere)

The notebook finds the nearest grid point within a tolerance (default: 0.01 degrees ≈ 1.1 km).

### Variable Extraction Workflow

1. For each variable, search for matching NetCDF files
2. Find nearest grid point to user-specified coordinates
3. Extract time series data from all files
4. Extract dates using time coordinate from NetCDF (if available) or filename
5. **Leap year handling**: Automatically reads all 366 days for leap years (e.g., 2024, 2028)
6. Combine and sort by date
7. Save as individual CSV files

### MET Format Specifications

APSIM MET format requires:
- **Header**: Metadata including latitude, longitude, tav, amp
- **Data columns**: year, day, radn, maxt, mint, rain, evap, vp, code
- **Required data**: maxt (from tasmax), mint (from tasmin), rain (from pr), radn (from rsds)
- **radn conversion**: rsds (W/m²) converted to radn (MJ/m²) - mandatory
- **Blank fields**: vp and code are left blank

### Unit Conversions

- **rsds** (W/m²) → **radn** (MJ/m²): Multiply by 0.0864
- **vp** and **code**: Left blank in MET format

### File Naming Conventions

All output files use coordinate-based naming:
- MET files: `{Model}_{Scenario}_{Lat}_{Lon}.met`
- CSV files: `{Model}_{Scenario}_{Lat}_{Lon}_{variable}.csv`

Example: `ACCESS_CM2_SSP245_-31.75_117.60.met`

### Model and Scenario Selection

Update the `MODEL` and `SCENARIO` variables in Section 1 to match your data folder structure.

Common scenarios:
- **SSP245**: Middle-of-the-road scenario
- **SSP585**: High-emissions scenario